In [1]:
from typing import Dict, List, Any
import pandas as pd

from doraemon import Doraemon
from sketch_of_thought import SoT
from reasoning_path_builder import ReasoningPath
from prompt_template_collection import PromptTemplate

df=pd.read_pickle('/kaggle/input/building-hotpotqa-injected-shuffled-dataset/hotpotqa_injected_shuffled_all_in_one_df.pkl')
df.shape

2025-06-23 09:07:38.386226: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750669658.663105      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750669658.743286      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


(1300, 5)

In [2]:
logger=Doraemon.get_logger(name=__name__, logfile='sot_on_hotpotqa_shuffled.log')

df.rename(columns={'answer':'ground_truth'}, inplace=True)


df['evidence_shuffled'] = df['evidence_shuffled'].apply(lambda x: ' '.join(x))
df['evidence_type']='Shuffled'

In [3]:
df['question'] = df.apply(lambda row: f"{row['question']} \nContext: {row['evidence_shuffled']}", axis=1)
logger.info(df.iloc[0].question)

2025-06-23 09:07:53,638 INFO VIVA Media AG changed it's name in 2004. What does their new acronym stand for? 
Context: A Gesellschaft mit beschränkter Haftung (] , abbreviated GmbH ] and also GesmbH in Austria) is a type of legal entity very common in Germany, Austria, Switzerland (where it is equivalent to a S.à r.l.) and Liechtenstein. VIVA Media GmbH (until 2004 "VIVA Media AG") is a music television network originating from Germany.


In [4]:
def final_prompt(question:str) -> str:
    return SoT.few_short_support(
        task_name='HOTPOTQA', 
        prompt=SoT.get_initialized_prompt(paradigm='conceptual_chaining'),
        few_short=SoT.get_few_short(task_name='HOTPOTQA'),
        question=question)

df['prompt']=df['question'].apply(final_prompt)

In [5]:
tasks: List[Dict[str, Any]]=df.head(1200).to_dict(orient='records')

temperatures = [i * 0.25 for i in range(9)]

final_tasks=[]
for t in tasks:
    for tp in temperatures:
        final_tasks.append((t, tp))

logger.info(len(final_tasks))

2025-06-23 09:07:53,695 INFO 10800


In [6]:
D:Dict[str, str] = PromptTemplate.sot_template_hotpotqa()
logger.info(len(D))

results:List[Dict]=ReasoningPath.run_parallel_inference(final_tasks, D, logger, 'sots', None)

2025-06-23 09:07:53,708 INFO 2
  0%|          | 8/10800 [00:08<3:55:39,  1.31s/it]2025-06-23 09:08:06,119 ERROR Error processing quetion VIVA Media AG changed it's name in 2004. What does their new acronym stand for? 
Context: A Gesellschaft mit beschränkter Haftung (] , abbreviated GmbH ] and also GesmbH in Austria) is a type of legal entity very common in Germany, Austria, Switzerland (where it is equivalent to a S.à r.l.) and Liechtenstein. VIVA Media GmbH (until 2004 "VIVA Media AG") is a music television network originating from Germany. with exception Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-05-01-preview have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 1 second. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit. For Free Account customers, upgrade to Pay as you Go here: https://aka

In [7]:
logger.info(results[0])

2025-06-23 14:12:29,572 INFO {'question': 'VIVA Media AG changed it\'s name in 2004. What does their new acronym stand for? \nContext: A Gesellschaft mit beschränkter Haftung (] , abbreviated GmbH ] and also GesmbH in Austria) is a type of legal entity very common in Germany, Austria, Switzerland (where it is equivalent to a S.à r.l.) and Liechtenstein. VIVA Media GmbH (until 2004 "VIVA Media AG") is a music television network originating from Germany.', 'context': 'A Gesellschaft mit beschränkter Haftung (] , abbreviated GmbH ] and also GesmbH in Austria) is a type of legal entity very common in Germany, Austria, Switzerland (where it is equivalent to a S.à r.l.) and Liechtenstein. VIVA Media GmbH (until 2004 "VIVA Media AG") is a music television network originating from Germany.', 'r_s': '<think>\nLet’s think through this step by step\n#VIVA Media AG → changed name to VIVA Media GmbH\n#GmbH → Gesellschaft mit beschränkter Haftung\n</think>\n\\boxed{Gesellschaft mit beschränkter Haft

In [8]:
filtered_results = [r for r in results if r is not None]

pd.DataFrame(filtered_results).to_pickle('sots_df.pkl')

In [9]:
df=pd.read_pickle('/kaggle/working/sots_df.pkl')
df.head()

,question,context,r_s,answer,ground_truth,tokens,temperature
0,VIVA Media AG changed it's name in 2004. What ...,"A Gesellschaft mit beschränkter Haftung (] , a...",<think>\nLet’s think through this step by step...,Gesellschaft mit beschränkter Haftung,Gesellschaft mit beschränkter Haftung,61,0.00
1,VIVA Media AG changed it's name in 2004. What ...,"A Gesellschaft mit beschränkter Haftung (] , a...",<think>\nLet’s think through this step by step...,Gesellschaft mit beschränkter Haftung,Gesellschaft mit beschränkter Haftung,63,0.25
2,VIVA Media AG changed it's name in 2004. What ...,"A Gesellschaft mit beschränkter Haftung (] , a...",<think>\nLet’s think through this step by step...,Gesellschaft mit beschränkter Haftung,Gesellschaft mit beschränkter Haftung,63,0.50
3,VIVA Media AG changed it's name in 2004. What ...,"A Gesellschaft mit beschränkter Haftung (] , a...",<think>\nLet’s think through this step by step...,Gesellschaft mit beschränkter Haftung,Gesellschaft mit beschränkter Haftung,69,0.75
4,VIVA Media AG changed it's name in 2004. What ...,"A Gesellschaft mit beschränkter Haftung (] , a...",<think>\nLet’s think through this step by step...,Gesellschaft mit beschränkter Haftung,Gesellschaft mit beschränkter Haftung,71,1.00
